<a href="https://colab.research.google.com/github/Thabet-Chaaouri/Efficient_fine_tuning/blob/main/Prompt_VS_Basic_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q peft transformers datasets accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, default_data_collator, get_linear_schedule_with_warmup
from peft import PromptTuningConfig, TaskType, PromptTuningInit, get_peft_model, PeftConfig, PeftModel
import torch
from datasets import load_dataset
import os
from torch.utils.data import DataLoader
from tqdm import tqdm
 

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_name = "bigscience/bloomz-560m"
dataset_name = "twitter_complaints"

text_column= "Tweet text"
label_column = "text_label"
max_length = 64
lr = 3e-2
num_epochs = 20 #50
batch_size = 8


#Load dataset

In [3]:
dataset = load_dataset("ought/raft", dataset_name)
dataset

Extracting data files:   0%|          | 0/11 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/50 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3399 [00:00<?, ? examples/s]

Dataset raft downloaded and prepared to /root/.cache/huggingface/datasets/ought___raft/twitter_complaints/1.1.0/79c4de1312c1e3730043f7db07179c914f48403101f7124e2fe336f6f54d9f84. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Tweet text', 'ID', 'Label'],
        num_rows: 50
    })
    test: Dataset({
        features: ['Tweet text', 'ID', 'Label'],
        num_rows: 3399
    })
})

#Preprocess data

In [5]:
classes = dataset["test"].features["Label"].names
dataset = dataset.map(lambda examples : {"text_label" : [classes[label] for label in examples["Label"]]}, batched=True, num_proc=1)
dataset

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/3399 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Tweet text', 'ID', 'Label', 'text_label'],
        num_rows: 50
    })
    test: Dataset({
        features: ['Tweet text', 'ID', 'Label', 'text_label'],
        num_rows: 3399
    })
})

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
target_max_length=max([len(tokenizer(label)["input_ids"]) for label in classes])
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

In [16]:
def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.pad_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = sample_input_ids + [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        )
        model_inputs["attention_mask"][i] = model_inputs["attention_mask"][i] + [0] * (max_length - len(sample_input_ids))
        labels["input_ids"][i] = label_input_ids + [-100] * (max_length - len(sample_input_ids))
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [17]:
processed_datasets = dataset.map(preprocess_function, batched=True, num_proc=1, remove_columns=dataset["train"].column_names, load_from_cache_file=False, desc="Runing tokenizer on dataset")

Runing tokenizer on dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Runing tokenizer on dataset:   0%|          | 0/3399 [00:00<?, ? examples/s]

In [18]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["test"]

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn = default_data_collator, pin_memory = True)
eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False, collate_fn = default_data_collator, pin_memory = True)

#Train

In [23]:
peft_config = PromptTuningConfig(task_type = TaskType.CAUSAL_LM, prompt_tuning_init=PromptTuningInit.TEXT, num_virtual_tokens=8,
                                prompt_tuning_init_text= "Classify if the tweet is a complaint or not:", tokenizer_name_or_path=model_name)

checkpoint_name = f"{dataset_name}_{model_name}_{peft_config.peft_type}_{peft_config.task_type}_v1.pt".replace("/", "_")

In [24]:
model = AutoModelForCausalLM.from_pretrained(model_name)
model =get_peft_model(model, peft_config)

In [25]:
print(model.print_trainable_parameters())

trainable params: 8192 || all params: 559222784 || trainable%: 0.0014648902430985358
None


In [26]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(optimizer=optimizer, num_warmup_steps=0,
                                               num_training_steps = (len(train_dataloader)*num_epochs))

In [27]:
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    # model.eval()
    # eval_loss = 0
    # eval_preds = []
    # for step, batch in enumerate(tqdm(eval_dataloader)):
    #     batch = {k: v.to(device) for k, v in batch.items()}
    #     with torch.no_grad():
    #         outputs = model(**batch)
    #     loss = outputs.loss
    #     eval_loss += loss.detach().float()
    #     eval_preds.extend(
    #         tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
    #     )

    # eval_epoch_loss = eval_loss / len(eval_dataloader)
    # eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=}")# {eval_ppl=} {eval_epoch_loss=}")

100%|██████████| 7/7 [00:05<00:00,  1.36it/s]


epoch=0: train_ppl=tensor(6.5875e+11, device='cuda:0') train_epoch_loss=tensor(27.2136, device='cuda:0')


100%|██████████| 7/7 [00:02<00:00,  2.86it/s]


epoch=1: train_ppl=tensor(2509.3027, device='cuda:0') train_epoch_loss=tensor(7.8278, device='cuda:0')


100%|██████████| 7/7 [00:02<00:00,  2.83it/s]


...

epoch=17: train_ppl=tensor(20.8906, device='cuda:0') train_epoch_loss=tensor(3.0393, device='cuda:0')


100%|██████████| 7/7 [00:02<00:00,  2.70it/s]


epoch=18: train_ppl=tensor(20.6651, device='cuda:0') train_epoch_loss=tensor(3.0284, device='cuda:0')


100%|██████████| 7/7 [00:02<00:00,  2.69it/s]

epoch=19: train_ppl=tensor(19.5618, device='cuda:0') train_epoch_loss=tensor(2.9736, device='cuda:0')


In [35]:
# correct = 0
# total = 0
# for pred, true in zip(eval_preds, dataset["test"]["text_label"]):
#     if pred.strip() == true.strip():
#         correct += 1
#     total += 1
# accuracy = correct / total * 100
# print(f"{accuracy=} % on the evaluation dataset")
# print(f"{eval_preds[:10]=}")
# print(f"{dataset['test']['text_label'][:10]=}")

#Share model

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("Thabet/bloomz-560m_PROMPT_TUNING_CAUSAL_LM", use_auth_token=True)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/33.5k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Thabet/bloomz-560m_PROMPT_TUNING_CAUSAL_LM/commit/f937d0debabf5232acf3ab8788940061508b4cb8', commit_message='Upload model', commit_description='', oid='f937d0debabf5232acf3ab8788940061508b4cb8', pr_url=None, pr_revision=None, pr_num=None)

#Load and infer

In [ ]:
peft_model_id = "Thabet/bloomz-560m_PROMPT_TUNING_CAUSAL_LM"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

In [33]:
inputs = tokenizer(
    f'{text_column} : {"@nationalgridus I am satisfied with your service"} Label : ',
    return_tensors="pt",
)

In [34]:
#model.to(device)

with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(
        input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=2, eos_token_id=3
    )
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

['Tweet text : @nationalgridus I am satisfied with your service Label : no complaint']


#Full finetuning

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name)
print(model.print_trainable_parameters())

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator(mixed_precision="fp16")

optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(optimizer=optimizer, num_warmup_steps=0,
                                               num_training_steps = (len(train_dataloader)*num_epochs))

model, optimizer, train_dataloader, eval_dataloader, scheduler = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader, scheduler)

In [ ]:
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)
        total_loss += loss.detach().float()
        #loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

In [ ]:
correct = 0
total = 0
for pred, true in zip(eval_preds, dataset["test"]["text_label"]):
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100
print(f"{accuracy=} % on the evaluation dataset")
print(f"{eval_preds[:10]=}")
print(f"{dataset['test']['text_label'][:10]=}")